In [1]:
import os

In [2]:
%pwd  #current dir

'/Users/fernandoecumbi/Desktop/IT course/Kidney-Disease-Classifciation-Deep-Learning-Project/research'

In [3]:
os.chdir("../") # Going back one folder

In [4]:

%pwd

'/Users/fernandoecumbi/Desktop/IT course/Kidney-Disease-Classifciation-Deep-Learning-Project'

In [3]:
# Entity file - return type of functions

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfg:
    root_dir: Path
    source_URl: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
# onfiguration file - to get the file
from cnnClassifier.constants import *
from cnnClassifier.utils.commom import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfg:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfg(
            root_dir=config.root_dir,
            source_URl=config.source.URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
# components file
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.commom import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfg):
        self.config = config

    def download_file(self) -> str:
        '''
        fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URl
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_data_ingestion", exist_ok= True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir )

            logger.info("f"Downloaded data from {dataset_url} inot file {zip_download_dir})

        except Exception as e:
          raise e


    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extract the zip file inot the data directory
        Function return None
        '''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [ ]:
# pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
